# DEPENDÊNCIAS

In [1]:
! pip install spacy
! pip install nltk
! pip install pandas
! pip install pdfplumber
! pip install chromadb
! pip install sentence-transformers
! pip install openpyxl
! pip install python-docx
! python -m spacy download pt_core_news_sm
! pip install langchain-text-splitters

     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
      --------------------------------------- 0.3/13.0 MB ? eta -:--:--
     - -------------------------------------- 0.5/13.0 MB 1.7 MB/s eta 0:00:08
     -- ------------------------------------- 0.8/13.0 MB 1.2 MB/s eta 0:00:10
     --- ------------------------------------ 1.0/13.0 MB 1.2 MB/s eta 0:00:10
     ---- ----------------------------------- 1.3/13.0 MB 1.1 MB/s eta 0:00:11
     ---- ----------------------------------- 1.6/13.0 MB 1.2 MB/s eta 0:00:10
     ---- ----------------------------------- 1.6/13.0 MB 1.2 MB/s eta 0:00:10
     ----- ---------------------------------- 1.8/13.0 MB 1.0 MB/s eta 0:00:11
     ----- ---------------------------------- 1.8/13.0 MB 1.0 MB/s eta 0:00:11
     ------ -------------------------------- 2.1/13.0 MB 989.5 kB/s eta 0:00:12
     ------- ------------------------------- 2.4/13.0 MB 984.3 kB/s eta 

# Carrega PDF

In [2]:
import pdfplumber

pdf_path = "./documents/sample_vangogh.pdf"

def load_pdf(filepath):
    with pdfplumber.open(filepath) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() or ""
    return text

raw_text = load_pdf(pdf_path)

print("Prévia do texto lido:\n")
print(raw_text[:500])


Prévia do texto lido:

Prólogo
Um coração fanático
Theo imaginou o pior. A mensagem dizia apenas que Vincent tinha “se ferido”.
Enquanto corria até a estação para pegar o próximo trem até Auvers, seus pensa-
mentos vagavam de trás para diante. Da última vez que havia recebido uma mensa-
gem parecida, era um telegrama de Paul Gauguin informando que Vincent estava
“gravemente enfermo”. Theo chegara à cidade meridional de Arles e encontrara o
irmão na enfermaria de um hospital, com a cabeça enfaixada e perdido em desvari


# PLN

## Lowercasing

In [3]:
text_lower = raw_text.lower()

print("Texto em lowercase (prévia):\n")
print(text_lower[:800])


Texto em lowercase (prévia):

prólogo
um coração fanático
theo imaginou o pior. a mensagem dizia apenas que vincent tinha “se ferido”.
enquanto corria até a estação para pegar o próximo trem até auvers, seus pensa-
mentos vagavam de trás para diante. da última vez que havia recebido uma mensa-
gem parecida, era um telegrama de paul gauguin informando que vincent estava
“gravemente enfermo”. theo chegara à cidade meridional de arles e encontrara o
irmão na enfermaria de um hospital, com a cabeça enfaixada e perdido em desvarios.
dessa vez, o que encontraria ao final da viagem?
em momentos assim — e eram muitos —, theo retornava às lembranças do
vincent que tinha conhecido no passado: um irmão mais velho ardoroso e irrequie-
to, mas também cheio de brincadeiras animadas, uma enorme afinidade e uma infi-
nita capacidade d


## Remover caracteres especiais

In [4]:
import re
import unicodedata

def remove_special_chars(text):
    # Normaliza acentos (NFKD)
    text = unicodedata.normalize("NFKD", text)
    text = text.encode("ASCII", "ignore").decode("utf-8")

    # Remove caracteres que não sejam letras ou espaço
    text = re.sub(r"[^a-z0-9.,\s]", " ", text)

    # Remove múltiplos espaços
    text = re.sub(r"\s+", " ", text)

    return text.strip()

text_clean = remove_special_chars(text_lower)

print("Texto sem caracteres especiais (prévia):\n")
print(text_clean[:800])


Texto sem caracteres especiais (prévia):

prologo um coracao fanatico theo imaginou o pior. a mensagem dizia apenas que vincent tinha se ferido. enquanto corria ate a estacao para pegar o proximo trem ate auvers, seus pensa mentos vagavam de tras para diante. da ultima vez que havia recebido uma mensa gem parecida, era um telegrama de paul gauguin informando que vincent estava gravemente enfermo. theo chegara a cidade meridional de arles e encontrara o irmao na enfermaria de um hospital, com a cabeca enfaixada e perdido em desvarios. dessa vez, o que encontraria ao final da viagem em momentos assim e eram muitos , theo retornava as lembrancas do vincent que tinha conhecido no passado um irmao mais velho ardoroso e irrequie to, mas tambem cheio de brincadeiras animadas, uma enorme afinidade e uma infi nita capacidade de admiracao. 


## Remover Stopwords

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Baixar recursos necessários (roda apenas 1 vez)
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")

stopwords_pt = set(stopwords.words("portuguese"))

tokens = word_tokenize(text_clean, language="portuguese")

tokens_no_stop = [t for t in tokens if t not in stopwords_pt]

print("Tokens sem stopwords (primeiros 30):\n")
print(tokens_no_stop[:30])


Tokens sem stopwords (primeiros 30):

['prologo', 'coracao', 'fanatico', 'theo', 'imaginou', 'pior', '.', 'mensagem', 'dizia', 'apenas', 'vincent', 'ferido', '.', 'enquanto', 'corria', 'ate', 'estacao', 'pegar', 'proximo', 'trem', 'ate', 'auvers', ',', 'pensa', 'mentos', 'vagavam', 'tras', 'diante', '.', 'ultima']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\notxy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\notxy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\notxy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Lemmatização

In [6]:
import spacy

# Carregar modelo de português
nlp = spacy.load("pt_core_news_sm")

def lemmatize_text(tokens):
    doc = nlp(" ".join(tokens))
    return [token.lemma_ for token in doc]

lemmas = " ".join(lemmatize_text(tokens_no_stop))

print("Lemmas (primeiros 30):\n")
print(lemmas[:300])


Lemmas (primeiros 30):

prologo coracao fanatico theo imaginar mau . mensagem dizer apenas Vincent ferido . enquanto correr ate estacao pegar proximo tr ate auver , pensar mento vagar tra diante . ultima vez haver receber menso gem parecir , telegrama paul gauguin informar vincent gravemente enfermo . theo chegar cidade me


## Stemmatização

In [7]:
# --- STEMMATIZAÇÃO (alternativa à lematização) ---
from nltk.stem.snowball import SnowballStemmer

# Criar stemmer para português
stemmer = SnowballStemmer("portuguese")

# Aplicar stemmatização nos tokens sem stopwords
stems = " ".join([stemmer.stem(token) for token in tokens_no_stop])

print("Stems (primeiros 30):\n")
print(stems[:300])


Stems (primeiros 30):

prolog coraca fanat the imagin pior . mensag diz apen vincent fer . enquant corr ate estaca peg proxim trem ate auvers , pens ment vag tras diant . ultim vez hav receb mens gem parec , telegram paul gauguin inform vincent gravement enferm . the cheg cidad meridional arles encontr irma enferm hospita


# Chunking


### Por Tamanho Fixo

In [8]:
# --- CHARACTER TEXT SPLITTER ---
from langchain_text_splitters import CharacterTextSplitter

char_splitter = CharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=10,
    separator=".",
    length_function=len
)

chunks_char = char_splitter.split_text(lemmas)

print(f"Total de chunks: {len(chunks_char)}")
print(chunks_char)


c:\Users\notxy\Documents\GitHub\walmir-p2-preparation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Created a chunk of size 223, which is longer than the specified 150
Created a chunk of size 162, which is longer than the specified 150
Created a chunk of size 166, which is longer than the specified 150
Created a chunk of size 215, which is longer than the specified 150
Created a chunk of size 167, which is longer than the specified 150
Created a chunk of size 165, which is longer than the specified 150
Created a chunk of size 159, which is longer than the specified 150
Created a chunk of size 254, which is longer than the specified 150
Created a chunk of size 233, which is longer than the specified 150
Created a chunk of size 186, which is longer than the specified 150
Created a chunk of size 166, which is

Total de chunks: 182
['prologo coracao fanatico theo imaginar mau . mensagem dizer apenas Vincent ferido', 'enquanto correr ate estacao pegar proximo tr ate auver , pensar mento vagar tra diante', 'ultima vez haver receber menso gem parecir , telegrama paul gauguin informar vincent gravemente enfermo', 'theo chegar cidade meridional arles encontrara irmao enfermaria hospital , cabeca enfaixar perder desvario', 'dessa vez , encontrar final viagem momento assim muito , theo retornar lembranco vincent conhecer passar irmao velho ardoroso Irrequie to , tamber cheio brincadeira animar , enorme afinidade infi nito capacidade admiracao', 'passeio infanciar campo mata redor cidade holandês zundert , onde haver nascir , Vincent apresentar beleza misterio natureza', 'inverno , Vincent ensinar patinar andar treno . verao , mostrar construir castelo trilha areiar', 'igreja domingo casa piano sala , cantar voz firme limper', 'quarto sotao divid , conversar ate tarde noite , criar irmao novo Vinculo

### Recursívo

In [9]:
# --- RECURSIVE CHARACTER TEXT SPLITTER ---
from langchain_text_splitters import RecursiveCharacterTextSplitter

recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=10,
    separators=["\n\n", "\n", ".", ",", " ", ""]
)

chunks_recursive = recursive_splitter.split_text(lemmas)

print(f"Total de chunks: {len(chunks_recursive)}")
print(chunks_recursive)


Total de chunks: 222
['prologo coracao fanatico theo imaginar mau . mensagem dizer apenas Vincent ferido', '. enquanto correr ate estacao pegar proximo tr ate auver , pensar mento vagar tra diante', '. ultima vez haver receber menso gem parecir , telegrama paul gauguin informar vincent gravemente enfermo', '. theo chegar cidade meridional arles encontrara irmao enfermaria hospital , cabeca enfaixar perder desvario', '. dessa vez , encontrar final viagem momento assim muito , theo retornar lembranco vincent conhecer passar irmao velho ardoroso Irrequie to', ', tamber cheio brincadeira animar , enorme afinidade infi nito capacidade admiracao', '. passeio infanciar campo mata redor cidade holandês zundert , onde haver nascir , Vincent apresentar beleza misterio natureza', '. inverno , Vincent ensinar patinar andar treno . verao , mostrar construir castelo trilha areiar', '. igreja domingo casa piano sala , cantar voz firme limper', '. quarto sotao divid , conversar ate tarde noite , criar

# Embedding

## Hugging Face SentenceTransformers

In [15]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

def embed_hf(chunks):
    vectors = model.encode(chunks, show_progress_bar=True)

    print(f"Dimensão: {vectors.shape}")
    print(vectors[:5])

    return vectors

# Testes
embedding = embed_hf(chunks_recursive)


Batches: 100%|██████████| 7/7 [00:01<00:00,  4.53it/s]

Dimensão: (222, 384)
[[ 0.08996075  0.08658748 -0.08757323 ...  0.06602014  0.06358144
  -0.0761409 ]
 [-0.06511153  0.00745593 -0.06131554 ...  0.06925161  0.05936948
   0.01263161]
 [-0.10004617  0.03950961 -0.04627548 ... -0.03181773 -0.07688483
  -0.055043  ]
 [ 0.03158804  0.09021249 -0.09092992 ...  0.01654903  0.00495357
  -0.12132753]
 [-0.02372601  0.06457057 -0.0517581  ...  0.11579664 -0.04759214
  -0.03465902]]


# Banco Vetorial

## Criação/Inserção ChromaDb

In [16]:
import chromadb

client = chromadb.Client()
if 'exemplo_colecao' in [col.name for col in client.list_collections()]:
    client.delete_collection("exemplo_colecao")

collection = client.get_or_create_collection(
    name="exemplo_colecao",
    embedding_function=None,
    # metadata={"hnsw:space": "cosine"}   # métrica mais comum
)
print(f"{collection.count()} documentos na coleção")

0 documentos na coleção


In [17]:
ids = [f"chunk_{i}" for i in range(len(chunks_recursive))]

collection.add(
    ids=ids,
    documents=chunks_recursive,
    embeddings=embedding
)

print("Embeddings inseridos com sucesso!")


Embeddings inseridos com sucesso!


## Busca Semântica

In [18]:
question = "Van Gogh?"
question_embedding = model.encode([question])

resultados = collection.query(
    query_embeddings=question_embedding.tolist(),
    n_results=3
)

print(f"\nPergunta: {question}")

for doc, score in zip(resultados['documents'][0], resultados['distances'][0]):
    print(f"(Score: {score}) - Documento: {doc}")


Pergunta: Van Gogh?
(Score: 0.6349847316741943) - Documento: . ninguem acreditar importancia biografiar fervor grande vincent van gogh
(Score: 0.6826222538948059) - Documento: . Willem , cinco ano idade , avo pintor vincent Willem van gogh
(Score: 0.7462995648384094) - Documento: . anna Cornelia carbentus casar reverer theodorus van gogh dia ceur limper , maio 1851 , haia , sede monarquia holandês , gur relato
